In [ ]:
import pandas as pd
import stanza

stanza.download('he', processors='tokenize,pos,lemma,depparse')
nlp = stanza.Pipeline('he', processors='tokenize,pos,lemma,depparse')



2025-03-17 14:58:24 WARNING: Language he package default expects mwt, which has been added
2025-03-17 14:58:24 INFO: Downloading these customized packages for language: he (Hebrew)...
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |
| pretrain  | htb     |

2025-03-17 14:58:24 INFO: File exists: /Users/pabloherrero/stanza_resources/he/tokenize/htb.pt.
2025-03-17 14:58:24 INFO: File exists: /Users/pabloherrero/stanza_resources/he/mwt/htb.pt.
2025-03-17 14:58:24 INFO: File exists: /Users/pabloherrero/stanza_resources/he/pos/htb.pt.
2025-03-17 14:58:24 INFO: File exists: /Users/pabloherrero/stanza_resources/he/lemma/htb.pt.
2025-03-17 14:58:24 INFO: File exists: /Users/pabloherrero/stanza_resources/he/depparse/htb.pt.
2025-03-17 14:58:24 INFO: File exists: /Users/pabloherrero/stanza_resources/he/pretrain/htb.pt.
2025-03-17 14:58:24 INFO: Finished downloading models and sa

# Test package

In [2]:
doc = nlp('הדר לא מותצה כלום להסתכל בטלוויזיה')
for sent in doc.sentences:
    for word in sent.words:
        print(word.text, word.lemma, word.upos, word.head, word.feats, word.deprel)

ה ה DET 2 Definite=Def|PronType=Art det
דר דר NOUN 4 Gender=Masc|Number=Sing nsubj
לא לא ADV 4 Polarity=Neg advmod
מותצה הותצה VERB 0 Gender=Masc|HebBinyan=HUFAL|Number=Sing|Person=1,2,3|VerbForm=Part|Voice=Pass root
כלום כלום ADV 4 None advmod
להסתכל הסתכל VERB 4 HebBinyan=HITPAEL|VerbForm=Inf xcomp
בטלוויזיה בטלוויזיה NOUN 6 Gender=Fem|Number=Sing obj


# Test on database

In [62]:
lippath = '/Users/pabloherrero/Documents/ManHatTan/data/processed/LIPSTICK/hebrew_db.lip'
lip = pd.read_csv(lippath)
tokens_list = list(lip['word_ll'].values)
tokens_list[:6]

['מקרה', 'מברשת שנים', 'עכביש', 'ענין', 'יחסית', 'להתבלבל']

In [ ]:
tokens_list = list(lip['word_ll'].values)

lexemes = []
for token in tokens_list:
    doc = nlp(token)
    # lernt/lernen<vblex><pri><p3><sg>,
    for sent in doc.sentences:
        lexeme_string = ''
        # print(len(sent.words))
        for word in sent.words:
            lexeme_string += word.text 
            if word.lemma:
                lexeme_string += '/' + word.lemma 
            if word.upos:
                lexeme_string += '<' + word.upos + '>'
            if word.feats:
                feats = word.feats.split('|')
                lexeme_string += ''.join(feat.split('=')[1] +'|' for feat in feats)
            if word.deprel != 'root':
                lexeme_string +=  ',' + word.deprel
        lexemes.append(lexeme_string)


In [142]:
# lexemes
lip.loc[:, 'lexeme_string'] = lexemes

In [144]:
lip.tail(8)

,p_recall,n_id,timestamp,delta,user_id,learning_language,ui_language,word_ll,word_ul,lexeme_string,...,mdt_history,mdt_correct,mrt_history,mrt_correct,wdt_history,wdt_correct,wrt_history,wrt_correct,speed,rebag
280,0.0,383,1738932382,0,pablo,iw,en,מפוחית,harmonica,מפוחית/מפוחית<PROPN>,...,0,0,0,0,0,0,0,0,0.0,False
281,0.0,56,1738932382,0,pablo,iw,en,חצוצרה,trumpet,חצוצרה/חצוצרה<PROPN>,...,0,0,0,0,0,0,0,0,0.0,False
282,0.0,115,1738932382,0,pablo,iw,en,מנגינה,tune,מנגינה/מנגינה<NOUN>Fem|Sing|,...,0,0,0,0,0,0,0,0,0.0,False
283,0.0,261,1738932382,0,pablo,iw,en,רמקול,speaker,רמקול/רמקול<NOUN>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0,False
284,0.0,234,1738932382,0,pablo,iw,en,מיקרופון,microphone,מיקרופון/מיקרופון<NOUN>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0,False
285,0.0,213,1738932382,0,pablo,iw,en,מתופף,drummer,"מתופף/התופף<VERB>Masc|PUAL|Sing|1,2,3|Part|Pass|",...,0,0,0,0,0,0,0,0,0.0,False
286,0.0,258,1738932382,0,pablo,iw,en,להקה,band,להקה/להקה<NOUN>Fem|Sing|,...,0,0,0,0,0,0,0,0,0.0,False
287,0.0,363,1738932382,0,pablo,iw,en,כינור,violin,כינור/כינור<NOUN>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0,False


## Write function:

In [145]:
def get_lexemes(lip):
    tokens_list = list(lip['word_ll'].values)

    lexemes = []
    for token in tokens_list:
        doc = nlp(token)
        # lernt/lernen<vblex><pri><p3><sg>,
        for sent in doc.sentences:
            lexeme_string = ''
            # print(len(sent.words))
            for word in sent.words:
                lexeme_string += word.text 
                if word.lemma:
                    lexeme_string += '/' + word.lemma 
                if word.upos:
                    lexeme_string += '<' + word.upos + '>'
                if word.feats:
                    feats = word.feats.split('|')
                    lexeme_string += ''.join(feat.split('=')[1] +'|' for feat in feats)
                if word.deprel != 'root':
                    lexeme_string +=  ',' + word.deprel
            lexemes.append(lexeme_string)
    return lexemes

In [146]:
def test_get_lexemes(lip):
    lexemes = get_lexemes(lip)
    assert len(lexemes) == len(lip)

In [148]:
lip.to_csv(lippath, index=False)

In [149]:
lip

,p_recall,n_id,timestamp,delta,user_id,learning_language,ui_language,word_ll,word_ul,lexeme_string,...,mdt_history,mdt_correct,mrt_history,mrt_correct,wdt_history,wdt_correct,wrt_history,wrt_correct,speed,rebag
0,1.0000,213,1738413429,52,pablo,iw,en,מקרה,case,מקרה/מקרה<NOUN>Masc|Sing|,...,0,0,2,2,3,3,0,0,0.1810,True
1,0.6250,236,1738413377,0,pablo,iw,en,מברשת שנים,toothbrush,מברשת/מברשת<NOUN>Cons|Fem|Sing|שנים/שנה<NOUN>F...,...,3,2,1,1,1,1,3,1,0.2779,True
2,1.0000,63,1738424677,11300,pablo,iw,en,עכביש,spider,עכביש/עכביש<PROPN>,...,1,1,2,2,0,0,2,2,0.1468,True
3,0.8333,191,1738413434,57,pablo,iw,en,ענין,matter,ענין/עניין<NOUN>Masc|Sing|,...,1,1,2,2,2,2,1,0,0.2385,True
4,0.8333,100,1738427203,13826,pablo,iw,en,יחסית,relatively,יחסית/יחסית<ADV>,...,2,2,1,1,1,1,2,1,0.2108,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0.0000,261,1738932382,0,pablo,iw,en,רמקול,speaker,רמקול/רמקול<NOUN>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0000,False
284,0.0000,234,1738932382,0,pablo,iw,en,מיקרופון,microphone,מיקרופון/מיקרופון<NOUN>Masc|Sing|,...,0,0,0,0,0,0,0,0,0.0000,False
285,0.0000,213,1738932382,0,pablo,iw,en,מתופף,drummer,"מתופף/התופף<VERB>Masc|PUAL|Sing|1,2,3|Part|Pass|",...,0,0,0,0,0,0,0,0,0.0000,False
286,0.0000,258,1738932382,0,pablo,iw,en,להקה,band,להקה/להקה<NOUN>Fem|Sing|,...,0,0,0,0,0,0,0,0,0.0000,False
